# DESI Fisher examples

In this notebook we will run some DESI forecasts.

In [1]:
import numpy as np

zedges = np.linspace(1.1, 1.6, 6)
zranges = list(zip(zedges[:-1], zedges[1:]))
nbars = [1079.9998, 1036.0009, 970.9995, 876.9994, 553.9985]  # nobj / deg^2 / dz
b0 = 0.84  # at z = 0

In [2]:
from cosmoprimo.fiducial import DESI
from desilike.theories.galaxy_clustering import (BAOPowerSpectrumTemplate, SimpleBAOWigglesTracerPowerSpectrumMultipoles, DampedBAOWigglesTracerPowerSpectrumMultipoles,
                                                 StandardPowerSpectrumTemplate, KaiserTracerPowerSpectrumMultipoles)
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable, CutskyFootprint, ObservablesCovarianceMatrix
from desilike.likelihoods.galaxy_clustering import ObservablesGaussianLikelihood, SNWeightedPowerSpectrumLikelihood
from desilike import Fisher, setup_logging

setup_logging()

## Let's compare simple isotropic BAO forecasts against some reference

### SN-weighted version

In [3]:
cosmo = DESI()
fo = cosmo.get_fourier()

covariances = []
for zrange, nbar in list(zip(zranges, nbars)):
    z = np.mean(zrange)
    r = 0.5
    sigmaper = (1 - r) * 12.4 * 0.758 * fo.sigma8_z(z, of='delta_cb') / 0.9
    f = fo.sigma8_z(z, of='theta_cb') / fo.sigma8_z(z, of='delta_cb')
    b1 = b0 * fo.sigma8_cb/fo.sigma8_z(z, of='delta_cb')  # prescription for linear bias
    params = {'b1': b1, 'sigmapar': (1. + f) * sigmaper, 'sigmaper': sigmaper}
    # Footprint, to get volume and shot noise; provided nbar is in deg^(-2) dz^(-1)
    footprint = CutskyFootprint(area=14000., zrange=zrange, nbar=nbar * np.diff(zrange), cosmo=cosmo)
    template = BAOPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qisoqap')
    #theory = DampedBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    theory = SimpleBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    likelihood = SNWeightedPowerSpectrumLikelihood(theories=theory, data=params, footprints=footprint, klim=(0.01, 0.5))

    fisher = Fisher(likelihood)
    precision = fisher(**params)
    covariances.append(precision.to_covariance(params=['qiso']))

# Let's compare against reference: within a few percent, not too bad!
refs = [0.7019, 0.7107, 0.7296, 0.7658, 1.0062]
print(100. * np.array([cov.std(params='qiso') for cov in covariances]) / np.array(refs)[:len(covariances)])

[000000.40] [0/1] 01-30 15:09  absl                      INFO     Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
[000000.40] [0/1] 01-30 15:09  absl                      INFO     Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
[000000.40] [0/1] 01-30 15:09  absl                      INFO     Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
[000000.41] [0/1] 01-30 15:09  absl                      INFO     Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
[000000.41] [0/1] 01-30 15:09  absl                      WARNING  No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


/home/adematti/Bureau/DESI/NERSC/cosmodesi/cosmoprimo/cosmoprimo/eisenstein_hu.py:23: UserWarning: EisensteinHuNoWiggleEngine cannot cope with massive neutrinos
  warnings.warn('{} cannot cope with massive neutrinos'.format(self.__class__.__name__))


[000001.24] [0/1] 01-30 15:09  Differentiation           INFO     Varied parameters: ['qiso', 'qap', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1'].
[000001.37] [0/1] 01-30 15:09  Differentiation           INFO     Using auto-differentiation for parameter qiso.
[000001.39] [0/1] 01-30 15:09  Differentiation           INFO     Using auto-differentiation for parameter qap.
[000001.44] [0/1] 01-30 15:09  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000001.47] [0/1] 01-30 15:09  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000001.49] [0/1] 01-30 15:09  Differentiation           INFO     Using auto-differentiation for parameter al0_-2.
[000001.52] [0/1] 01-30 15:09  Differentiation           INFO     Using auto-differentiation for parameter al0_-1.
[000001.54] [0/1] 01-30 15:09  Differentiation           INFO     Using auto-differentiation for parameter al0_0.
[

[000050.83] [0/1] 01-30 15:10  Differentiation           INFO     Using auto-differentiation for parameter al0_1.
[000050.85] [0/1] 01-30 15:10  Differentiation           INFO     Using auto-differentiation for parameter al2_-3.
[000050.88] [0/1] 01-30 15:10  Differentiation           INFO     Using auto-differentiation for parameter al2_-2.
[000050.91] [0/1] 01-30 15:10  Differentiation           INFO     Using auto-differentiation for parameter al2_-1.
[000050.93] [0/1] 01-30 15:10  Differentiation           INFO     Using auto-differentiation for parameter al2_0.
[000050.95] [0/1] 01-30 15:10  Differentiation           INFO     Using auto-differentiation for parameter al2_1.
[000050.97] [0/1] 01-30 15:10  Differentiation           INFO     Varied parameters: ['qiso', 'qap', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1'].
[000051.00] [0/1] 01-30 15:10  Differentiation           INFO     Using finite-differentiation for parameter 

[000101.53] [0/1] 01-30 15:11  Differentiation           INFO     Using finite-differentiation for parameter qap.
[000104.59] [0/1] 01-30 15:11  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000105.16] [0/1] 01-30 15:11  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000105.74] [0/1] 01-30 15:11  Differentiation           INFO     Using auto-differentiation for parameter al0_-2.
[000106.40] [0/1] 01-30 15:11  Differentiation           INFO     Using auto-differentiation for parameter al0_-1.
[000107.11] [0/1] 01-30 15:11  Differentiation           INFO     Using auto-differentiation for parameter al0_0.
[000107.72] [0/1] 01-30 15:11  Differentiation           INFO     Using auto-differentiation for parameter al0_1.
[000108.33] [0/1] 01-30 15:11  Differentiation           INFO     Using auto-differentiation for parameter al2_-3.
[000108.92] [0/1] 01-30 15:11  Differentiation           INFO     Using auto-differenti

### Comparison against multipole compression

In [4]:
covariances2 = []
for zrange, nbar in list(zip(zranges, nbars)):
    z = np.mean(zrange)
    r = 0.5
    sigmaper = (1 - r) * 12.4 * 0.758 * fo.sigma8_z(z, of='delta_cb') / 0.9
    f = fo.sigma8_z(z, of='theta_cb') / fo.sigma8_z(z, of='delta_cb')
    params = {'b1': b0 * fo.sigma8_cb/fo.sigma8_z(z, of='delta_cb'), 'sigmapar': (1. + f) * sigmaper, 'sigmaper': sigmaper}
    
    footprint = CutskyFootprint(area=14000., zrange=zrange, nbar=nbar * np.diff(zrange), cosmo=cosmo)
    template = BAOPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qisoqap')
    #theory = DampedBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    theory = SimpleBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    observable = TracerPowerSpectrumMultipolesObservable(data=params,  # data can be a dictionary of parameters
                                                         # fit monopole and quadrupole, between 0.02 and 0.5 h/Mpc, with 0.005 h/Mpc steps
                                                         klim={0: [0.01, 0.5, 0.005], 2: [0.01, 0.5, 0.005], 4: [0.01, 0.5, 0.005]},
                                                         theory=theory)
    covariance = ObservablesCovarianceMatrix(observable, footprints=footprint, resolution=5)  # Gaussian covariance matrix
    likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=covariance(**params))
    fisher = Fisher(likelihood)
    precision = fisher(**params)
    covariances2.append(precision.to_covariance(params=['qiso']))

# multipole compression is very close to optimal...
print(np.array([cov.std(params='qiso') for cov in covariances2]) / np.array([cov.std(params='qiso') for cov in covariances]))

/home/adematti/anaconda3/envs/cosmodesi/lib/python3.9/site-packages/numpy/lib/polynomial.py:1329: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


[000132.41] [0/1] 01-30 15:11  Differentiation           INFO     Varied parameters: ['qiso', 'qap', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1', 'al4_-3', 'al4_-2', 'al4_-1', 'al4_0', 'al4_1'].
[000132.45] [0/1] 01-30 15:11  Differentiation           INFO     Using auto-differentiation for parameter qiso.
[000132.48] [0/1] 01-30 15:11  Differentiation           INFO     Using auto-differentiation for parameter qap.
[000132.51] [0/1] 01-30 15:11  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000132.55] [0/1] 01-30 15:11  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000132.58] [0/1] 01-30 15:11  Differentiation           INFO     Using auto-differentiation for parameter al0_-2.
[000132.61] [0/1] 01-30 15:11  Differentiation           INFO     Using auto-differentiation for parameter al0_-1.
[000132.64] [0/1] 01-30 15:11  Differentiation           INFO     U

[000145.01] [0/1] 01-30 15:12  Differentiation           INFO     Using auto-differentiation for parameter al0_1.
[000145.06] [0/1] 01-30 15:12  Differentiation           INFO     Using auto-differentiation for parameter al2_-3.
[000145.11] [0/1] 01-30 15:12  Differentiation           INFO     Using auto-differentiation for parameter al2_-2.
[000145.16] [0/1] 01-30 15:12  Differentiation           INFO     Using auto-differentiation for parameter al2_-1.
[000145.21] [0/1] 01-30 15:12  Differentiation           INFO     Using auto-differentiation for parameter al2_0.
[000145.26] [0/1] 01-30 15:12  Differentiation           INFO     Using auto-differentiation for parameter al2_1.
[000145.30] [0/1] 01-30 15:12  Differentiation           INFO     Using auto-differentiation for parameter al4_-3.
[000145.35] [0/1] 01-30 15:12  Differentiation           INFO     Using auto-differentiation for parameter al4_-2.
[000145.40] [0/1] 01-30 15:12  Differentiation           INFO     Using auto-differ

[000164.83] [0/1] 01-30 15:12  Differentiation           INFO     Using auto-differentiation for parameter al4_-1.
[000164.86] [0/1] 01-30 15:12  Differentiation           INFO     Using auto-differentiation for parameter al4_0.
[000164.89] [0/1] 01-30 15:12  Differentiation           INFO     Using auto-differentiation for parameter al4_1.
[000164.91] [0/1] 01-30 15:12  Differentiation           INFO     Varied parameters: ['qiso', 'qap', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1', 'al4_-3', 'al4_-2', 'al4_-1', 'al4_0', 'al4_1'].
[000164.94] [0/1] 01-30 15:12  Differentiation           INFO     Using finite-differentiation for parameter qiso.
[000164.97] [0/1] 01-30 15:12  Differentiation           INFO     Using finite-differentiation for parameter qap.
[000165.19] [0/1] 01-30 15:12  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000165.24] [0/1] 01-30 15:12  Differentiation           INFO    

## BAO + full-shape forecasts

In [5]:
zranges, nbars = zranges[:1], nbars[:1]
cosmo = DESI()
fo = cosmo.get_fourier()

covariances_bao, covariances_bao_highk, covariances_fs_lowk, covariances_bao_fs = [], [], [], []
for zrange, nbar in list(zip(zranges, nbars)):
    z = np.mean(zrange)
    r = 0.5
    sigmaper = (1 - r) * 12.4 * 0.758 * fo.sigma8_z(z, of='delta_cb') / 0.9
    f = fo.sigma8_z(z, of='theta_cb') / fo.sigma8_z(z, of='delta_cb')
    b1 = b0 * fo.sigma8_cb/fo.sigma8_z(z, of='delta_cb')
    params = {'b1': b1, 'sigmapar': (1. + f) * sigmaper, 'sigmaper': sigmaper}
    footprint = CutskyFootprint(area=14000., zrange=zrange, nbar=nbar * np.diff(zrange), cosmo=cosmo)
    template = BAOPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qparqper')
    #theory = DampedBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    theory = SimpleBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    likelihood = SNWeightedPowerSpectrumLikelihood(theories=theory, data=params, footprints=footprint, klim=(0.01, 0.5))

    fisher = Fisher(likelihood)
    precision = fisher(**params)
    covariances_bao.append(precision.to_covariance(params=['qpar', 'qper']))
    
    likelihood.init.update(klim=(0.1, 0.5))
    fisher = Fisher(likelihood)
    precision_bao = fisher(**params)
    covariances_bao_highk.append(precision_bao.to_covariance(params=['qpar', 'qper']))
    
    params = {'b1': b1}
    template = StandardPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qparqper')
    theory = KaiserTracerPowerSpectrumMultipoles(template=template)
    likelihood = SNWeightedPowerSpectrumLikelihood(theories=theory, data=params, footprints=footprint, klim=(0.01, 0.1))
    fisher = Fisher(likelihood)
    precision_fs = fisher(**params)
    covariances_fs_lowk.append(precision_bao.to_covariance(params=['qpar', 'qper']))
    
    covariances_bao_fs.append((precision_bao + precision_fs).to_covariance(params=['qpar', 'qper']))

[000178.41] [0/1] 01-30 15:12  Differentiation           INFO     Varied parameters: ['qpar', 'qper', 'b1', 'al0_-3', 'al0_-2', 'al0_-1', 'al0_0', 'al0_1', 'al2_-3', 'al2_-2', 'al2_-1', 'al2_0', 'al2_1'].
[000178.45] [0/1] 01-30 15:12  Differentiation           INFO     Using auto-differentiation for parameter qpar.
[000178.50] [0/1] 01-30 15:12  Differentiation           INFO     Using auto-differentiation for parameter qper.
[000178.53] [0/1] 01-30 15:12  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000178.57] [0/1] 01-30 15:12  Differentiation           INFO     Using auto-differentiation for parameter al0_-3.
[000178.60] [0/1] 01-30 15:12  Differentiation           INFO     Using auto-differentiation for parameter al0_-2.
[000178.63] [0/1] 01-30 15:12  Differentiation           INFO     Using auto-differentiation for parameter al0_-1.
[000178.66] [0/1] 01-30 15:12  Differentiation           INFO     Using auto-differentiation for parameter al0_0.

[000235.69] [0/1] 01-30 15:13  Differentiation           INFO     Using auto-differentiation for parameter f.
[000238.31] [0/1] 01-30 15:13  Differentiation           INFO     Using auto-differentiation for parameter b1.
[000240.69] [0/1] 01-30 15:13  Differentiation           INFO     Using auto-differentiation for parameter sn0.


In [6]:
# 40% improvement using the full shape < 0.1 h/Mpc
for cov_bao, cov_bao_fs in zip(covariances_bao, covariances_bao_fs):
    print(cov_bao.std('qpar') / cov_bao_fs.std('qpar'), cov_bao.std('qper') / cov_bao_fs.std('qper'))

1.468265060878483 1.4038779318010828
